In [1]:
import pickle
import os
import networkx as nx
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import torch
import torch.nn as nn
import pyflagser
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# data loader

In [2]:
import pickle
with open('34 bus review/WeekdayScenarios.pkl', 'rb') as g:
    P0Data = pickle.load(g)
N_Senario=len(P0Data)

# generate Label

In [3]:
Class=[]
for i in range(300):
    Class.append(P0Data[i]['Stations Attacked'])
print(Class)

[[1], [1], [], [2], [1], [1], [], [1], [0], [], [2], [2], [1], [1], [1], [0], [2], [2], [2], [2], [1], [2], [1], [0], [], [], [2], [1], [1], [2], [1], [1], [], [0], [1], [2], [0], [0], [2], [], [0], [1], [0], [0], [], [0], [0], [0], [0], [1], [0], [], [2], [0], [], [1], [], [0], [0], [], [1], [1], [0], [2], [2], [0], [1], [], [0], [0], [], [2], [2], [1], [1], [], [1], [2], [0], [0], [], [2], [0], [0], [2], [0], [], [1], [1], [1], [0], [0], [1], [0], [1], [2], [1], [2], [2], [2], [1], [0], [0], [], [2], [1], [1], [2], [1], [1], [1], [0], [0], [], [0], [], [0], [1], [0], [], [0], [0], [2], [], [2], [0], [1], [], [2], [], [1], [1], [1], [0], [1], [1], [1], [2], [0], [1], [0], [0], [2], [], [], [], [2], [2], [], [1], [0], [1], [1], [], [0], [2], [], [], [2], [1], [], [2], [2], [2], [], [2], [2], [], [1], [2], [0], [2], [0], [1], [2], [2], [1], [0], [1], [], [2], [0], [1], [1], [], [], [], [], [], [2], [2], [], [1], [], [0], [], [1], [0], [0], [1], [0], [2], [1], [0], [2], [], [1], [], [2],

In [4]:
def classify_sublist(sublist):
    if sublist == [0]:
        return 0  # Class 0 for []
    elif sublist == [1]:
        return 1  # Class 1 for [0]
    elif sublist == [2]:
        return 2  # Class 2 for [1]
    elif sublist == []:
        return 3  # Class 3 for [2]
    else:
        return -7  # For sublists not matching any class (shouldn't happen)

In [5]:
Label=[]
Label = [classify_sublist(sublist) for sublist in Class]

print(Label)

[1, 1, 3, 2, 1, 1, 3, 1, 0, 3, 2, 2, 1, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 0, 3, 3, 2, 1, 1, 2, 1, 1, 3, 0, 1, 2, 0, 0, 2, 3, 0, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 3, 2, 0, 3, 1, 3, 0, 0, 3, 1, 1, 0, 2, 2, 0, 1, 3, 0, 0, 3, 2, 2, 1, 1, 3, 1, 2, 0, 0, 3, 2, 0, 0, 2, 0, 3, 1, 1, 1, 0, 0, 1, 0, 1, 2, 1, 2, 2, 2, 1, 0, 0, 3, 2, 1, 1, 2, 1, 1, 1, 0, 0, 3, 0, 3, 0, 1, 0, 3, 0, 0, 2, 3, 2, 0, 1, 3, 2, 3, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 0, 2, 3, 3, 3, 2, 2, 3, 1, 0, 1, 1, 3, 0, 2, 3, 3, 2, 1, 3, 2, 2, 2, 3, 2, 2, 3, 1, 2, 0, 2, 0, 1, 2, 2, 1, 0, 1, 3, 2, 0, 1, 1, 3, 3, 3, 3, 3, 2, 2, 3, 1, 3, 0, 3, 1, 0, 0, 1, 0, 2, 1, 0, 2, 3, 1, 3, 2, 0, 3, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 1, 3, 3, 1, 3, 3, 1, 3, 2, 2, 1, 2, 2, 3, 1, 2, 3, 1, 1, 0, 3, 0, 2, 1, 0, 1, 0, 3, 3, 1, 2, 1, 2, 2, 2, 1, 1, 3, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 0, 3, 0, 2, 3, 0, 0, 2, 3, 2, 3, 0, 2, 2, 2, 0, 3, 0, 0, 3, 3, 1, 1, 0, 3, 2, 1, 2]


# label statistics

In [6]:
from collections import Counter

counter = Counter(Label)

for key, value in counter.items():
    print(f"Element {key} appears {value} times")

Element 1 appears 90 times
Element 3 appears 65 times
Element 2 appears 72 times
Element 0 appears 73 times


# make the node voltage as time series format

In [25]:
def Average(lst):
    return sum(lst) / len(lst)
def make_timeseries(P0Data,scenario, total_time_step,Nodes,Edges):
    time_series_voltage=[]
    time_series_BFlow=[]
    for time_step in range(total_time_step):
        voltage=[]
        B_flow=[]
        for node in Nodes:
            node_voltage=P0Data[scenario]['Bus Voltages'][node][time_step]
            #voltage.append(P0Data[scenario]['Bus Voltages'][node][time_step].tolist())
            voltage.append(Average(node_voltage))
        time_series_voltage.append(voltage)
        for edge in Edges:
            B_flow.append([P0Data[scenario]['Branch flows'][edge][time_step]])
        time_series_BFlow.append(B_flow)
    return time_series_voltage, time_series_BFlow

# construct graph G

In [7]:
node = P0Data[0]['Bus Voltages'].keys()
Edge = P0Data[0]['Branch flows'].keys()
# Initialize an empty graph
G = nx.Graph()  # or nx.DiGraph() for a directed graph

# Add nodes (convert dict_keys to a list if needed)
G.add_nodes_from(list(node))

# Add edges
G.add_edges_from(Edge)

In [8]:
N_Scenario=300
N=list(G.nodes)
E=list(G.edges)

# construct mean node voltage (only baseline)

In [26]:
N_Scenario=300
N=list(G.nodes)
E=list(G.edges)
feat = [make_timeseries(P0Data,i, 288,N,E)[0] for i in tqdm(range(N_Scenario), desc="Processing")]


Processing: 100%|█████████████████████████████| 300/300 [00:07<00:00, 40.36it/s]


In [ ]:
from tqdm import tqdm

# extract topological features

In [16]:
def print_stat(train_acc, test_acc,pre,rec,f1):
    argmax=np.argmax(train_acc)
    best_result=test_acc[argmax]
    train_acc=np.max(train_acc)
    test_acc=np.max(test_acc)
    print(f'Train accuracy = {train_acc*100:.2f}%,Test Accuracy = {test_acc*100:.2f}%,precision = {np.max(pre)*100:.2f}%,Recall = {np.max(rec)*100:.2f}%,f1-score = {np.max(f1)*100:.2f}%\n')
    return test_acc

def Average(lst):
    return sum(lst) / len(lst)
def make_timeseries(scenario, total_time_step):
    time_series_voltage=[]
    time_series_BFlow=[]
    for time_step in range(total_time_step):
        voltage=[]
        B_flow=[]
        for node in N:
            node_voltage=P0Data[scenario]['Bus Voltages'][node][time_step]
            #voltage.append(P0Data[scenario]['Bus Voltages'][node][time_step].tolist())
            voltage.append(Average(node_voltage))
        time_series_voltage.append(voltage)
        for edge in E:
            B_flow.append([P0Data[scenario]['Branch flows'][edge][time_step]])
        time_series_BFlow.append(B_flow)
    return time_series_voltage, time_series_BFlow

def Topo_Fe_TimeSeries_MP(TS_voltage, TS_branchFlow, F_voltage,F_Flow):
    betti_0=[]
    for k in range(len(TS_voltage)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_voltage[k]
        for y in Voltage:
            AverageVoltage.append(y)
        #AverageVoltage = [i * 100 for i in AverageVoltage]
        BranchFlow=[]
        Branch_Flow=TS_branchFlow[k]
        for j  in range(len(Branch_Flow)):
            BranchFlow.append(Branch_Flow[j][0])

        for p in range(len(F_voltage)):
            Active_node_v=np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            for q in range(len(F_Flow)):
                #if AverageVoltage[p]> F_voltage[p] and BranchFlow[q]> F_Flow[q]:
                #n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
                n_active=Active_node_v.copy()
                #print(n_active)
                G = nx.DiGraph()
                G.add_nodes_from(n_active)
                indices = np.where(np.array(BranchFlow) > F_Flow[q])[0].tolist()
                for s in indices:
                    a=int(N.index(E[s][0]))
                    b=int(N.index(E[s][1]))
                    if a in n_active and b in n_active:
                        G.add_edge(a, b)
                    #n_active.append(int(N.index(E[s][0])))
                    #n_active.append(int(N.index(E[s][1])))
                #Active_node=np.unique(n_active)
                #print(G.edges())
                if (len(n_active)==0):
                    fec.append(0)
                else:
                    #b=A[Active_node,:][:,Active_node]
                    Adj = nx.to_numpy_array(G)
                    my_flag=pyflagser.flagser_unweighted(Adj, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                    x = my_flag["betti"]
                    fec.append(x[0])
                n_active.clear()
            Active_node_v.clear()
        betti_0.append(fec)
    return betti_0


In [17]:
# voltage_threshold = np.array([0.34, 0.35, 0.94, 0.98, 0.99, 1.01, 1.02, 1.03, 1.04, 1.05])

# BFlow_threshold = np.array([8.8e-01, 1.75e+00, 7.24e+00, 2.453e+01, 5.928e+01,
#                             1.3625e+02, 3.9623e+02, 5.0359e+02, 1.29343e+03,
#                             1.33055e+03, 1.60132e+03, 1.66341e+03, 1.73374e+03])
voltage_threshold=np.array([1.04, 1.03, 1.02,1.0, 0.99, 0.98, 0.96,0.95,0.94,0.93,0.91, 0.35, 0.34])
# BFlow_threshold=np.array([1.73374e+03, 1.66341e+03, 1.33055e+03, 
#                                 5.03590e+02, 1.36250e+02, 5.92800e+01, 2.45300e+01, 
#                                 7.24000e+00, 8.80000e-01])

BFlow_threshold=np.array([1.73374e+03, 1.33055e+03, 
                                5.03590e+02, 1.36250e+02, 5.92800e+01, 2.45300e+01, 
                                7.24000e+00, 8.80000e-01])
def extract_topological_features(graph_id):
    time_series=make_timeseries(graph_id,time_step)
    betti=Topo_Fe_TimeSeries_MP(time_series[0],time_series[1], voltage_threshold,BFlow_threshold)
    return betti

def process_graph(graph_id):
    # Function to extract topological features from a single graph
    return extract_topological_features(graph_id)

In [15]:
time_step=288
feat_mp_500=[]
for i in tqdm(range(300),desc='processing'):
    bet=process_graph(i)
    feat_mp_500.append(bet)


processing: 100%|█████████████████████████████| 300/300 [48:28<00:00,  9.70s/it]


# save the topological features

In [16]:
with open('EV_34bus_betti0_review.data', 'wb') as f:
    pickle.dump(feat_mp_500, f)

# load the topological features

In [10]:
#first 200 scenarios
with open('EV_34bus_betti0_review.data', 'rb') as f:
        data_mp = pickle.load(f)

In [11]:

from sklearn.preprocessing import MinMaxScaler
X0=np.array(data_mp)
y=Label

# X=[]
# for i in range(N_Senario):

#     scaler = MinMaxScaler()

# # Fit scaler to data and transform data
#     X.append(scaler.fit_transform(X0[i]))

# Convert data to PyTorch tensors
X = torch.tensor(X0, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

num_samples = N_Senario
num_timesteps = 288
#num_features = len(F_voltage)*len(F_Flow)
num_features = len(X[0][0])
num_classes = len(np.unique(y))

In [19]:
print(num_classes)

4


In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score


# Define the Transformer model with Batch Normalization
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_heads, n_layers, num_timesteps):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, num_timesteps, hidden_dim))
        self.transformer = nn.Transformer(d_model=hidden_dim, nhead=n_heads, num_encoder_layers=n_layers, num_decoder_layers=n_layers)
        self.fc = nn.Linear(hidden_dim * num_timesteps, output_dim)  # Flatten the output of the transformer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, src):
        src_emb = self.embedding(src) + self.positional_encoding[:, :src.size(1), :]
        src_emb = src_emb.permute(1, 0, 2)  # (seq_len, batch, feature)
        transformer_output = self.transformer.encoder(src_emb)
        transformer_output = transformer_output.permute(1, 0, 2).contiguous().view(src.size(0), -1)  # Flatten
        transformer_output = self.dropout(transformer_output)
        predictions = self.fc(transformer_output)
        return predictions


def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

# Define input and output dimensions (example placeholders)
input_dim = num_features
hidden_dim = 64
output_dim = num_classes
n_heads = 2
n_layers = 2
num_timesteps = 288  # Adjust based on your sequence length

# Initialize model, loss function, and optimizer
model = TransformerClassifier(input_dim, hidden_dim, output_dim, n_heads, n_layers, num_timesteps)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=23)

# Create DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Lists to store metrics
train_losses = []
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []

# Train the model
reset_weights(model)
model.train()
for epoch in range(400):
    epoch_train_loss = 0
    correct_train = 0
    total_train = 0
    
    # Training loop
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        epoch_train_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total_train += y_batch.size(0)
        correct_train += (predicted == y_batch).sum().item()

    avg_train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Evaluate on the test set
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            _, predicted = torch.max(output, 1)
            total_test += y_batch.size(0)
            correct_test += (predicted == y_batch).sum().item()

            # Store predictions and targets for metrics
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(y_batch.cpu().numpy())

    test_accuracy = correct_test / total_test
    test_accuracies.append(test_accuracy)
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(all_targets, all_preds, average='weighted',zero_division=0)
    recall = recall_score(all_targets, all_preds, average='weighted',zero_division=0)
    f1 = f1_score(all_targets, all_preds, average='weighted',zero_division=0)

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    model.train()  # Switch back to training mode

    # Print metrics for this epoch
    print(f'Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}%, Test Accuracy = {test_accuracy:.4f}%')
    print(f'Precision = {precision:.2f}, Recall = {recall:.4f}, F1-Score = {f1:.4f}')
# Save the trained model
model_path = '34Bus_transformer_classifier.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

print('=============\n finished training\n=============')
acc=print_stat(train_accuracies, test_accuracies,precisions,recalls,f1_scores)


/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1: Train Loss = 5.2768, Train Accuracy = 0.2625%, Test Accuracy = 0.3667%
Precision = 0.13, Recall = 0.3667, F1-Score = 0.1967
Epoch 2: Train Loss = 4.9137, Train Accuracy = 0.2583%, Test Accuracy = 0.3667%
Precision = 0.13, Recall = 0.3667, F1-Score = 0.1967
Epoch 3: Train Loss = 2.8494, Train Accuracy = 0.2500%, Test Accuracy = 0.3667%
Precision = 0.13, Recall = 0.3667, F1-Score = 0.1967
Epoch 4: Train Loss = 2.3108, Train Accuracy = 0.2417%, Test Accuracy = 0.3667%
Precision = 0.13, Recall = 0.3667, F1-Score = 0.1967
Epoch 5: Train Loss = 2.0938, Train Accuracy = 0.1917%, Test Accuracy = 0.3667%
Precision = 0.13, Recall = 0.3667, F1-Score = 0.1967
Epoch 6: Train Loss = 1.8257, Train Accuracy = 0.2583%, Test Accuracy = 0.2333%
Precision = 0.05, Recall = 0.2333, F1-Score = 0.0883
Epoch 7: Train Loss = 1.8038, Train Accuracy = 0.2333%, Test Accuracy = 0.2333%
Precision = 0.05, Recall = 0.2333, F1-Score = 0.0883
Epoch 8: Train Loss = 1.8080, Train Accuracy = 0.2417%, Test Accuracy

Epoch 63: Train Loss = 0.4477, Train Accuracy = 0.8625%, Test Accuracy = 0.5500%
Precision = 0.71, Recall = 0.5500, F1-Score = 0.5490
Epoch 64: Train Loss = 0.3545, Train Accuracy = 0.8708%, Test Accuracy = 0.8500%
Precision = 0.89, Recall = 0.8500, F1-Score = 0.8531
Epoch 65: Train Loss = 0.2059, Train Accuracy = 0.9375%, Test Accuracy = 0.7667%
Precision = 0.81, Recall = 0.7667, F1-Score = 0.7517
Epoch 66: Train Loss = 0.1969, Train Accuracy = 0.9292%, Test Accuracy = 0.7667%
Precision = 0.80, Recall = 0.7667, F1-Score = 0.7437
Epoch 67: Train Loss = 0.1861, Train Accuracy = 0.9417%, Test Accuracy = 0.7167%
Precision = 0.77, Recall = 0.7167, F1-Score = 0.7118
Epoch 68: Train Loss = 0.2719, Train Accuracy = 0.9000%, Test Accuracy = 0.8000%
Precision = 0.89, Recall = 0.8000, F1-Score = 0.7733
Epoch 69: Train Loss = 0.3091, Train Accuracy = 0.8750%, Test Accuracy = 0.6500%
Precision = 0.73, Recall = 0.6500, F1-Score = 0.6169
Epoch 70: Train Loss = 0.3912, Train Accuracy = 0.8667%, Test 

Epoch 124: Train Loss = 0.0412, Train Accuracy = 0.9958%, Test Accuracy = 0.8500%
Precision = 0.89, Recall = 0.8500, F1-Score = 0.8453
Epoch 125: Train Loss = 0.0380, Train Accuracy = 0.9875%, Test Accuracy = 0.9333%
Precision = 0.95, Recall = 0.9333, F1-Score = 0.9323
Epoch 126: Train Loss = 0.0350, Train Accuracy = 0.9875%, Test Accuracy = 0.9333%
Precision = 0.94, Recall = 0.9333, F1-Score = 0.9308
Epoch 127: Train Loss = 0.0443, Train Accuracy = 0.9917%, Test Accuracy = 0.9167%
Precision = 0.92, Recall = 0.9167, F1-Score = 0.9133
Epoch 128: Train Loss = 0.0212, Train Accuracy = 0.9917%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9665
Epoch 129: Train Loss = 0.0479, Train Accuracy = 0.9917%, Test Accuracy = 0.9167%
Precision = 0.93, Recall = 0.9167, F1-Score = 0.9153
Epoch 130: Train Loss = 0.0633, Train Accuracy = 0.9792%, Test Accuracy = 0.8833%
Precision = 0.90, Recall = 0.8833, F1-Score = 0.8803
Epoch 131: Train Loss = 0.0672, Train Accuracy = 0.9792

Epoch 185: Train Loss = 0.1318, Train Accuracy = 0.9625%, Test Accuracy = 0.9333%
Precision = 0.93, Recall = 0.9333, F1-Score = 0.9333
Epoch 186: Train Loss = 0.0499, Train Accuracy = 0.9750%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9664
Epoch 187: Train Loss = 0.0239, Train Accuracy = 0.9958%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9651
Epoch 188: Train Loss = 0.0291, Train Accuracy = 0.9958%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9650
Epoch 189: Train Loss = 0.0117, Train Accuracy = 1.0000%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9650
Epoch 190: Train Loss = 0.0231, Train Accuracy = 0.9792%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9485
Epoch 191: Train Loss = 0.1238, Train Accuracy = 0.9875%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9650
Epoch 192: Train Loss = 0.0544, Train Accuracy = 0.9750

Epoch 246: Train Loss = 0.1188, Train Accuracy = 0.9750%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9668
Epoch 247: Train Loss = 0.1385, Train Accuracy = 0.9708%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9667
Epoch 248: Train Loss = 1.1765, Train Accuracy = 0.8083%, Test Accuracy = 0.2000%
Precision = 0.39, Recall = 0.2000, F1-Score = 0.1289
Epoch 249: Train Loss = 4.2990, Train Accuracy = 0.2750%, Test Accuracy = 0.2500%
Precision = 0.09, Recall = 0.2500, F1-Score = 0.1161
Epoch 250: Train Loss = 2.3162, Train Accuracy = 0.2458%, Test Accuracy = 0.1500%
Precision = 0.02, Recall = 0.1500, F1-Score = 0.0391
Epoch 251: Train Loss = 1.7705, Train Accuracy = 0.2792%, Test Accuracy = 0.4833%
Precision = 0.27, Recall = 0.4833, F1-Score = 0.3357
Epoch 252: Train Loss = 2.2729, Train Accuracy = 0.2750%, Test Accuracy = 0.3667%
Precision = 0.13, Recall = 0.3667, F1-Score = 0.1967
Epoch 253: Train Loss = 2.1509, Train Accuracy = 0.3292

Epoch 307: Train Loss = 0.0764, Train Accuracy = 0.9625%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9507
Epoch 308: Train Loss = 0.1123, Train Accuracy = 0.9417%, Test Accuracy = 0.8833%
Precision = 0.90, Recall = 0.8833, F1-Score = 0.8810
Epoch 309: Train Loss = 0.1247, Train Accuracy = 0.9500%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9510
Epoch 310: Train Loss = 0.1198, Train Accuracy = 0.9542%, Test Accuracy = 0.9333%
Precision = 0.94, Recall = 0.9333, F1-Score = 0.9333
Epoch 311: Train Loss = 0.0865, Train Accuracy = 0.9667%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9507
Epoch 312: Train Loss = 0.1159, Train Accuracy = 0.9583%, Test Accuracy = 0.9167%
Precision = 0.92, Recall = 0.9167, F1-Score = 0.9165
Epoch 313: Train Loss = 0.0364, Train Accuracy = 0.9875%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9507
Epoch 314: Train Loss = 0.0596, Train Accuracy = 0.9667

Epoch 368: Train Loss = 0.0685, Train Accuracy = 0.9792%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9507
Epoch 369: Train Loss = 0.0087, Train Accuracy = 1.0000%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9667
Epoch 370: Train Loss = 0.0251, Train Accuracy = 0.9917%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9507
Epoch 371: Train Loss = 0.0266, Train Accuracy = 0.9875%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9667
Epoch 372: Train Loss = 0.0203, Train Accuracy = 0.9958%, Test Accuracy = 0.9667%
Precision = 0.97, Recall = 0.9667, F1-Score = 0.9667
Epoch 373: Train Loss = 0.0111, Train Accuracy = 0.9958%, Test Accuracy = 0.9500%
Precision = 0.95, Recall = 0.9500, F1-Score = 0.9488
Epoch 374: Train Loss = 0.0200, Train Accuracy = 0.9875%, Test Accuracy = 0.9167%
Precision = 0.92, Recall = 0.9167, F1-Score = 0.9165
Epoch 375: Train Loss = 0.0246, Train Accuracy = 0.9917

# test on new dataset

In [13]:
import pickle
with open('34 bus review/TestScenarios.pkl', 'rb') as g:
    P0Data = pickle.load(g)
N_Senario=len(P0Data)

In [31]:
P0Data[3]

{'Anomaly': 1,
 'Stations Attacked': [1],
 'Type': 4,
 'Bus Voltages': {'sourcebus': [array([1.04999862, 1.04999931, 1.04999928]),
   array([1.04999865, 1.04999934, 1.04999931]),
   array([1.04999871, 1.04999941, 1.04999938]),
   array([1.0499989 , 1.04999961, 1.04999957]),
   array([1.04999894, 1.04999965, 1.04999961]),
   array([1.04999899, 1.04999968, 1.04999963]),
   array([1.04999903, 1.04999971, 1.04999966]),
   array([1.04999907, 1.04999975, 1.0499997 ]),
   array([1.04999911, 1.04999978, 1.04999973]),
   array([1.04999915, 1.04999982, 1.04999977]),
   array([1.0499992 , 1.04999985, 1.0499998 ]),
   array([1.04999921, 1.04999989, 1.04999984]),
   array([1.0499991 , 1.04999978, 1.04999973]),
   array([1.04999911, 1.04999979, 1.04999974]),
   array([1.04999914, 1.04999982, 1.04999977]),
   array([1.04999927, 1.04999994, 1.04999989]),
   array([1.04999928, 1.04999995, 1.0499999 ]),
   array([1.04999929, 1.04999996, 1.04999991]),
   array([1.04999931, 1.04999997, 1.04999992]),
   ar

In [32]:
time_step=288
bet=process_graph(3) # extract topo features

In [33]:
new_X=torch.tensor([bet],dtype=torch.float32)# convert to tensor

In [ ]:
len(new_X[0])

In [34]:
# Re-initialize the model with the same architecture
model = TransformerClassifier(input_dim, hidden_dim, output_dim, n_heads, n_layers, num_timesteps)

# Load the saved model weights
model.load_state_dict(torch.load('34Bus_transformer_classifier.pth'))
model.eval()  # Set the model to evaluation mode

# Test on new data
with torch.no_grad():
    new_output = model(new_X)  # new_X should be preprocessed like the original training data
    _, new_predictions = torch.max(new_output, 1)

print("Predictions on new data: station under attack is", new_predictions)

Predictions on new data: station under attack is tensor([1])


/var/folders/k8/fv2pzvcn5p77dgds_c16lsl80000gn/T/ipykernel_1091/2972270458.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('34Bus_transf